In [3]:
import datasets
import glob

In [2]:
train_dataset = {"text":[], "label":[]}

In [3]:
math_file_paths = glob.glob("../data/examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        train_dataset["text"].append(f.read())
        train_dataset["label"].append(1)

In [4]:
math_file_paths = glob.glob("../data/not_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        train_dataset["text"].append(f.read())
        train_dataset["label"].append(0)

In [5]:
train_dataset = datasets.Dataset.from_dict(train_dataset)

In [6]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2779
})

In [7]:
test_dataset = {"text":[], "label":[]}

In [8]:
math_file_paths = glob.glob("../data/test_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        test_dataset["text"].append(f.read())
        test_dataset["label"].append(1)

In [9]:
math_file_paths = glob.glob("../data/test_not_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        test_dataset["text"].append(f.read())
        test_dataset["label"].append(0)

In [10]:
test_dataset = datasets.Dataset.from_dict(test_dataset)

In [11]:
datadict = datasets.DatasetDict({
    "train":train_dataset,
    "test":test_dataset
})

In [196]:
datadict.save_to_disk("./dataset")

Saving the dataset (0/1 shards):   0%|          | 0/2779 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/387 [00:00<?, ? examples/s]

# 新增

In [22]:
import sys
import os

sys.path.append('../')

from paper_markdown_text_classifier import extract_text

In [23]:
datadict = datasets.load_dataset("ranWang/test_paper_textClassifier")

Found cached dataset parquet (/home/ran/.cache/huggingface/datasets/ranWang___parquet/ranWang--test_paper_textClassifier-c89de31317ad5101/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
train_dataset = datadict["train"]

In [25]:
train_dataset

Dataset({
    features: ['text', 'label', 'file_path'],
    num_rows: 9536
})

In [26]:
all_file_name = {row["file_path"].split("/")[-1] for row in train_dataset}

In [27]:
from pathlib import Path
from tqdm import tqdm
import os


counterexamples = Path("./negative_file").glob("**/*.doc*")

column = {"text":[], "label":[], "file_path": []}
for file_path in tqdm(list(counterexamples)):
    _, ext = os.path.splitext(file_path)
    file_name = str(file_path).split("/")[-1]
    
    if file_name in all_file_name:
        continue
        
    try:
        text = extract_text(str(file_path), ext)
    except:
        continue
    
    column["text"].append(text)
    column["label"].append(0)
    column["file_path"].append(str(file_path))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5644/5644 [00:01<00:00, 4920.52it/s]


In [28]:
from pathlib import Path

counterexamples = Path("./positive_file").glob("**/*.doc*")

for file_path in tqdm(list(counterexamples)):
    _, ext = os.path.splitext(file_path)
    file_name = str(file_path).split("/")[-1]
    
    if file_name in all_file_name:
        continue
        
    try:
        text = extract_text(str(file_path), ext)
    except:
        continue
        
    column["text"].append(text)
    column["label"].append(1)
    column["file_path"].append(str(file_path))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1795/1795 [00:31<00:00, 56.50it/s]


In [29]:
train_dataset = datasets.concatenate_datasets([train_dataset, datasets.Dataset.from_dict(column)])

In [30]:
train_dataset

Dataset({
    features: ['text', 'label', 'file_path'],
    num_rows: 10191
})

In [31]:
datadict["train"] = train_dataset

In [32]:
datadict

DatasetDict({
    test: Dataset({
        features: ['text', 'label', 'file_path'],
        num_rows: 387
    })
    train: Dataset({
        features: ['text', 'label', 'file_path'],
        num_rows: 10191
    })
})

In [33]:
datadict.save_to_disk("./new_classification_datasets")

Saving the dataset (0/1 shards):   0%|          | 0/387 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10191 [00:00<?, ? examples/s]